In [1]:
import cntk as C
from PIL import Image
import os
import numpy as np
import urllib
from scipy.misc import imsave

try:
    from urllib.request import urlretrieve, urlopen
except ImportError:
    from urllib import urlretrieve, urlopen

try:
    C.device.try_set_default_device(C.device.gpu(0))
except:
    print("GPU unavailable. Using CPU instead.")

In [2]:
#prefer our default path for the data
data_dir = os.path.join("data", "vihance")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

#folder with images to be evaluated
example_folder = os.path.join(data_dir, "example_images")
if not os.path.exists(example_folder):
    os.makedirs(example_folder)

#folders with resulting images
results_folder = os.path.join(data_dir, "example_results")
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
    
#folders with VDSR models
models_dir = os.path.join(data_dir, "Models")
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

In [3]:
from keras.models import load_model

if not os.path.isfile(os.path.join(models_dir, "VDSR.model")):
    print("Downloading VDSR model...")
    urlretrieve("https://www.cntk.ai/Models/SuperResolution/VDSR.model", os.path.join(models_dir, "VDSR.model"))
else:
    print("Using cached VDSR model")

print("Loading models...")
VDSR_model = C.load_model(os.path.join(models_dir, "VDSR.model"))
VDSR_trained_model = load_model(os.path.join(models_dir, "vdsr_trained.hdf5"))

print("Loaded pretrained models.")

C:\Users\Aldi\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using CNTK backend


Using cached VDSR model
Loading models...
Loaded pretrained models.


In [4]:
#filename - relative path of image being processed
#outfile - relative path of the image which will be saved

def evaluate_pretrained(filename, outfile):
    img = Image.open(filename)
    model = VDSR_model
    output_dims = 64

    #upscaling coefficient
    coef = 2

    #at each step, we will evaluate subpatch (x : x + range_x, y : y + range_y) of original image
    #patch by patch, we will resolve the whole image
    range_x = output_dims // coef
    range_y = output_dims // coef

    #how many bounding pixels from resulting patch should be excluded?
    #this is important because boundaries tend to be predicted less accurately
    offset = output_dims // 10

    #after we evaluate a subpatch, how much we move down/right to get the next one
    #we subtract offset to cover those pixels which were boundary in the previous subpatch
    step_x = range_x - offset
    step_y = range_y - offset

    #pre-magnify picture if needed
    img = img.resize((coef * img.width, coef * img.height), Image.BICUBIC)

    #if the current image is being cleared up with no further uspcaling,
    #set coef to 1 and other parameters accordingly
    result = np.zeros((img.height, img.width, 3))
    range_x = output_dims
    range_y = output_dims
    step_x = range_x - 2 * offset
    step_y = range_y - 2 * offset
    coef = 1

    rect = np.array(img, dtype = np.float32)

    #if the image is too small for some models to work on it, pad it with zeros
    if(rect.shape[0] < range_y):
        pad = np.zeros((range_y - rect.shape[0], rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 0).astype(dtype = np.float32)

    if(rect.shape[1] < range_x):
        pad = np.zeros((rect.shape[0], range_x - rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 1).astype(dtype = np.float32)

    x = 0
    y = 0

    #take subpatch by subpatch and resolve them to get the final image result
    while(y < img.width):
        x = 0
        while(x < img.height):
            rgb_patch = rect[x : x + range_x, y : y + range_y]
            rgb_patch = rgb_patch[..., [2, 1, 0]]
            rgb_patch = np.ascontiguousarray(np.rollaxis(rgb_patch, 2))
            pred = np.squeeze(model.eval({model.arguments[0] : [rgb_patch]}))

            img1 = np.ascontiguousarray(rgb_patch.transpose(2, 1, 0))
            img2 = np.ascontiguousarray(pred.transpose(2, 1, 0))

            #if model predicts residual image,
            #scale back the prediction and add to starting patch
            #otherwise just scale back
            img2 = 255.0 * img2 + img1

            # make sure img2 is C Contiguous as we just transposed it
            img2 = np.ascontiguousarray(img2)
            #make sure no pixels are outside [0, 255] interval
            for _ in range(2):
                img2 = C.relu(img2).eval()
                img2 = np.ones(img2.shape) * 255.0 - img2

            rgb = img2[..., ::-1]
            patch = rgb.transpose(1, 0, 2)

            #fill in the pixels in the middle of the subpatch
            #don't fill those within offset range to the boundary
            for h in range(coef * x + offset, coef * x + output_dims - offset):
                for w in range(coef * y + offset, coef * y + output_dims - offset):
                    for col in range(0, 3):
                        result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #pad top
            if(x == 0):
                for h in range(offset):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h][w - coef * y][col]

            #pad left
            if(y == 0):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(offset):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w][col]

            #pad bottom
            if(x == img.height - range_x):
                for h in range(coef * img.height - offset, coef * img.height):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #pad right
            if(y == img.width - range_y):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(coef * img.width - offset, coef * img.width):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #reached bottom of image
            if(x == img.height - range_x):
                break
            #next step by x, we must not go out of bounds
            x = min(x + step_x, img.height - range_x)

        #reached right edge of image
        if(y == img.width - range_x):
            break
        #next step by y, we must not go out of bounds
        y = min(y + step_y, img.width - range_x)

    result = np.ascontiguousarray(result)

    #save result
    imsave(outfile, result.astype(np.uint8))

In [5]:
#filename - relative path of image being processed
#outfile - relative path of the image which will be saved

def evaluate_trained(filename, outfile):
    model = VDSR_trained_model
    output_dims = 64
    img = Image.open(filename)

    #upscaling coefficient
    coef = 2

    #at each step, we will evaluate subpatch (x : x + range_x, y : y + range_y) of original image
    #patch by patch, we will resolve the whole image
    range_x = output_dims // coef
    range_y = output_dims // coef

    #how many bounding pixels from resulting patch should be excluded?
    #this is important because boundaries tend to be predicted less accurately
    offset = output_dims // 10

    #after we evaluate a subpatch, how much we move down/right to get the next one
    #we subtract offset to cover those pixels which were boundary in the previous subpatch
    step_x = range_x - offset
    step_y = range_y - offset

    img = img.resize((coef * img.width, coef * img.height), Image.BICUBIC)

    result = np.zeros((img.height, img.width, 3))
    range_x = output_dims
    range_y = output_dims
    step_x = range_x - 2 * offset
    step_y = range_y - 2 * offset
    coef = 1

    rect = np.array(img, dtype = np.float32)

    #if the image is too small for some models to work on it, pad it with zeros
    if(rect.shape[0] < range_y):
        pad = np.zeros((range_y - rect.shape[0], rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 0).astype(dtype = np.float32)

    if(rect.shape[1] < range_x):
        pad = np.zeros((rect.shape[0], range_x - rect.shape[1], rect.shape[2]))
        rect = np.concatenate((rect, pad), axis = 1).astype(dtype = np.float32)

    x = 0
    y = 0
    
    #take subpatch by subpatch and resolve them to get the final image result
    while(y < img.width):
        x = 0
        while(x < img.height):
            rgb_patch = rect[x : x + range_x, y : y + range_y]
            rgb_patch = rgb_patch[..., [2, 1, 0]]
            rgb_patch = np.ascontiguousarray(np.rollaxis(rgb_patch, 2))
            pred = model.predict(rgb_patch.transpose(2,1,0).reshape(1,64,64,3))

            img2 = pred.reshape(64,64,3)

            # make sure img2 is C Contiguous as we just transposed it
            img2 = np.ascontiguousarray(img2)
            #make sure no pixels are outside [0, 255] interval
            for _ in range(2):
                img2 = C.relu(img2).eval()
                img2 = np.ones(img2.shape) * 255.0 - img2

            rgb = img2[..., ::-1]
            patch = rgb.transpose(1, 0, 2)

            #fill in the pixels in the middle of the subpatch
            #don't fill those within offset range to the boundary
            for h in range(coef * x + offset, coef * x + output_dims - offset):
                for w in range(coef * y + offset, coef * y + output_dims - offset):
                    for col in range(0, 3):
                        result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #pad top
            if(x == 0):
                for h in range(offset):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h][w - coef * y][col]

            #pad left
            if(y == 0):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(offset):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w][col]

            #pad bottom
            if(x == img.height - range_x):
                for h in range(coef * img.height - offset, coef * img.height):
                    for w in range(coef * y, coef * y + output_dims):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #pad right
            if(y == img.width - range_y):
                for h in range(coef * x, coef * x + output_dims):
                    for w in range(coef * img.width - offset, coef * img.width):
                        for col in range(0, 3):
                            result[h][w][col] = patch[h - coef * x][w - coef * y][col]

            #reached bottom of image
            if(x == img.height - range_x):
                break
            #next step by x, we must not go out of bounds
            x = min(x + step_x, img.height - range_x)

        #reached right edge of image
        if(y == img.width - range_x):
            break
        #next step by y, we must not go out of bounds
        y = min(y + step_y, img.width - range_x)

    result = np.ascontiguousarray(result)

    #save result
    imsave(outfile, result.astype(np.uint8))

In [6]:
save_folder = os.path.join(results_folder, "bicubic")

#upscale by bicubic and save for reference
for entry in os.listdir(example_folder):
    filename = os.path.join(example_folder, entry)

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    img = Image.open(filename)
    out.save(os.path.join(save_folder, entry))

In [7]:
import time

save_folder = os.path.join(results_folder, "VDSR_trained_results")

#loop through every image in example_folder
for entry in os.listdir(example_folder):
    filename = os.path.join(example_folder, entry)

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    outfile = os.path.join(save_folder, entry)

    t = time.time()
    print("Now creating: " + outfile)

    evaluate_trained(filename, outfile)

    print("Time is {} ms".format(time.time() - t))
    
save_folder = os.path.join(results_folder, "VDSR_pretrained_results")

#loop through every image in example_folder
for entry in os.listdir(example_folder):
    filename = os.path.join(example_folder, entry)

    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    outfile = os.path.join(save_folder, entry)

    t = time.time()
    print("Now creating: " + outfile)

    evaluate_pretrained(filename, outfile)

    print("Time is {} ms".format(time.time() - t))

Now creating: data\vihance\example_results\VDSR_trained_results\1525878424968.jpg


C:\Users\Aldi\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\cntk\core.py:82: RuntimeWarning: data is not C contiguous; rearrange your data/computation to avoid costly data conversions
  RuntimeWarning)


Time is 7.287409782409668 ms
Now creating: data\vihance\example_results\VDSR_trained_results\253027.jpg
Time is 3.8238871097564697 ms
Now creating: data\vihance\example_results\VDSR_pretrained_results\1525878424968.jpg
Time is 5.588998079299927 ms
Now creating: data\vihance\example_results\VDSR_pretrained_results\253027.jpg
Time is 3.4904215335845947 ms
